In [ ]:
from poor_man_lakehouse.spark.builder import retrieve_current_spark_session
from poor_man_lakehouse.config import settings

spark = retrieve_current_spark_session()

In [ ]:
spark.sql("SHOW CATALOGS").show()
spark.sql(f"show schemas in {settings.CATALOG}").show()
spark.sql("show schemas in spark_catalog").show()
spark.sql(f"SHOW  tables in {settings.CATALOG}.default").show()

In [ ]:
import polars as pl
import datetime
from pyspark.sql import DataFrame

data = pl.DataFrame(
    {
        "datetime": [
            datetime.datetime(2023, 1, 1, 12, 0),
            datetime.datetime(2023, 1, 2, 12, 0),
            datetime.datetime(2023, 1, 3, 12, 0),
        ],
        "symbol": ["AAPL", "GOOGL", "MSFT"],
        "bid": [150.0, 2800.0, 300.0],
        "ask": [151.0, 2805.0, 305.0],
        "details": [
            {"created_by": "user1"},
            {"created_by": "user2"},
            {"created_by": None},
        ],
    },
)

spark_df: DataFrame = spark.createDataFrame(data.to_pandas())


In [ ]:
spark_df.write.format("iceberg").mode("overwrite").saveAsTable(
    f"{settings.CATALOG}.default.prova", mode="overwrite"
)

In [ ]:
spark.sql(f"""SELECT * FROM {settings.CATALOG}.default.prova""").show()

## Testing LakeSail

To test Lakesail, it's necessary to restart the kernel since the sparkSession was already created and we need a clean one.

In [ ]:
from pysail.spark import SparkConnectServer
from pyspark.sql import SparkSession

server = SparkConnectServer()
server.start()
address = server.listening_address
if address is None:
    raise RuntimeError("Failed to start Spark Connect server")
_, port = address

spark = SparkSession.builder.remote(f"sc://localhost:{port}").getOrCreate()


In [ ]:
path = "s3a://warehouse/test_parquet"
df = spark.createDataFrame([(1, "Alice"), (2, "Bob")], schema="id INT, name STRING")
df.write.parquet(path)

df = spark.read.parquet(path)
